In [58]:
## Basic import of of Image

In [101]:
import gdal
import matplotlib
from matplotlib import pyplot as plt
import random
import numpy as np
from PIL import Image
import scipy.misc
import tifffile
import rasterio
import cv2

In [102]:
def ReadBilFile(bil, band):
    gdal.GetDriverByName('EHdr').Register()
    img = gdal.Open(bil)
    band = img.GetRasterBand(band)
    data = band.ReadAsArray()
    return data

In [103]:
## For all the RS images in the dataset
## Chop each study area on 20 images randomly into 256*256*6

In [152]:
!mkdir CIAdata
!mkdir CIAdata/test
!mkdir CIAdata/train
!mkdir CIAdata/val

mkdir: CIAdata: File exists
mkdir: CIAdata/test: File exists
mkdir: CIAdata/train: File exists
mkdir: CIAdata/val: File exists


In [132]:
## Set up path
modis_path_arr = []
landsat_path_arr = []


modis_path_arr.append("./MODIS/2001_281_08oct/CIA_mod_2001_281.dat")
landsat_path_arr.append('./Landsat/2001_281_08oct/CIA_ls_2001_281.dat')

modis_path_arr.append("./MODIS/2001_290_17oct/CIA_mod_2001_290.dat")
landsat_path_arr.append('./Landsat/2001_290_17oct/CIA_ls_2001_290.dat')

modis_path_arr.append("./MODIS/2001_306_02nov/CIA_mod_2001_306.dat")
landsat_path_arr.append('./Landsat/2001_306_02nov/CIA_ls_2001_306.dat')

modis_path_arr.append("./MODIS/2001_313_09nov/CIA_mod_2001_313.dat")
landsat_path_arr.append('./Landsat/2001_313_09nov/CIA_ls_2001_313.dat')

modis_path_arr.append("./MODIS/2001_329_25nov/CIA_mod_2001_329.dat")
landsat_path_arr.append('./Landsat/2001_329_25nov/CIA_ls_2001_329.dat')

modis_path_arr.append("./MODIS/2001_338_04dec/CIA_mod_2001_338.dat")
landsat_path_arr.append('./Landsat/2001_338_04dec/CIA_ls_2001_338.dat')

modis_path_arr.append("./MODIS/2002_005_05jan/CIA_mod_2002_005.dat")
landsat_path_arr.append('./Landsat/2002_005_05jan/CIA_ls_2002_005.dat')

modis_path_arr.append("./MODIS/2002_012_12jan/CIA_mod_2002_012.dat")
landsat_path_arr.append('./Landsat/2002_012_12jan/CIA_ls_2002_012.dat')

modis_path_arr.append("./MODIS/2002_044_13feb/CIA_mod_2002_044.dat")
landsat_path_arr.append('./Landsat/2002_044_13feb/CIA_ls_2002_044.dat')

modis_path_arr.append("./MODIS/2002_053_22feb/CIA_mod_2002_053.dat")
landsat_path_arr.append('./Landsat/2002_053_22feb/CIA_ls_2002_053.dat')

modis_path_arr.append("./MODIS/2002_069_10mar/CIA_mod_2002_069.dat")
landsat_path_arr.append('./Landsat/2002_069_10mar/CIA_ls_2002_069.dat')

modis_path_arr.append("./MODIS/2002_076_17mar/CIA_mod_2002_076.dat")
landsat_path_arr.append('./Landsat/2002_076_17mar/CIA_ls_2002_076.dat')

modis_path_arr.append("./MODIS/2002_092_02apr/CIA_mod_2002_092.dat")
landsat_path_arr.append('./Landsat/2002_092_02apr/CIA_ls_2002_092.dat')

modis_path_arr.append("./MODIS/2002_101_11apr/CIA_mod_2002_101.dat")
landsat_path_arr.append('./Landsat/2002_101_11apr/CIA_ls_2002_101.dat')

modis_path_arr.append("./MODIS/2002_108_18apr/CIA_mod_2002_108.dat")
landsat_path_arr.append('./Landsat/2002_108_18apr/CIA_ls_2002_108.dat')

modis_path_arr.append("./MODIS/2002_117_27apr/CIA_mod_2002_117.dat")
landsat_path_arr.append('./Landsat/2002_117_27apr/CIA_ls_2002_117.dat')

modis_path_arr.append("./MODIS/2002_124_04may/CIA_mod_2002_124.dat")
landsat_path_arr.append('./Landsat/2002_124_04may/CIA_ls_2002_124.dat')

In [151]:
## test with first 3 bands for example
## 20 images per sample for training
## 5 images per sample for testing
## 5 images per sample for validation

curr_train = 1
curr_test = 1
curr_val = 1

for index in range(0, len(landsat_path_arr)):
    modis_path = modis_path_arr[index]
    landsat_path = landsat_path_arr[index]
    print(index)
    for sampleid in range(0,30):
        ## make it into 256*512*6
        ## MODIS first
        ##dummy = np.zeros((256,512,6), dtype=np.int16)
        ## 3 band result
        dummy = np.zeros((256,512,3), dtype=np.int16)


        ## Get the band value for modis
        modis = gdal.Open(modis_path)
        modis_1 = modis.GetRasterBand(1).ReadAsArray()
        modis_2 = modis.GetRasterBand(2).ReadAsArray()
        modis_3 = modis.GetRasterBand(3).ReadAsArray()
        #modis_4 = modis.GetRasterBand(4).ReadAsArray()
        #modis_5 = modis.GetRasterBand(5).ReadAsArray()
        #modis_6 = modis.GetRasterBand(6).ReadAsArray()
        ## Get the band value for Landsat
        
        landsat = gdal.Open(landsat_path)
        landsat_1 = landsat.GetRasterBand(1).ReadAsArray()
        landsat_2 = landsat.GetRasterBand(2).ReadAsArray()
        landsat_3 = landsat.GetRasterBand(3).ReadAsArray()
        #landsat_4 = landsat.GetRasterBand(4).ReadAsArray()
        #landsat_5 = landsat.GetRasterBand(5).ReadAsArray()
        #landsat_6 = landsat.GetRasterBand(6).ReadAsArray()

        
        ## Get the dimension of the data
        (x,y) = ReadBilFile(landsat_path, 1).shape
        
        ## Get a random start
        start_x = random.randint(0, x-256)
        start_y = random.randint(0, y-256)

        for i in range(0, 256):
            for j in range(0, 256):
                ## band1
                dummy[i][j][0] = modis_1[i+start_x][j+start_y]
                dummy[i][j+256][0] = landsat_1[i+start_x][j+start_y]
                ## band2
                dummy[i][j][1] = modis_2[i+start_x][j+start_y]
                dummy[i][j+256][1] = landsat_2[i+start_x][j+start_y]
                ## band3
                dummy[i][j][2] = modis_3[i+start_x][j+start_y]
                dummy[i][j+256][2] = landsat_3[i+start_x][j+start_y]
                ## band4
                ##dummy[i][j][3] = modis_4[i+start_x][j+start_y]
                ##dummy[i][j+256][3] = landsat_4[i+start_x][j+start_y]
                ## band5
                ##dummy[i][j][4] = modis_5[i+start_x][j+start_y]
                ##dummy[i][j+256][4] = landsat_5[i+start_x][j+start_y]
                ## band6
                ##dummy[i][j][5] = modis_6[i+start_x][j+start_y]
                ##dummy[i][j+256][5] = landsat_6[i+start_x][j+start_y]
                ## into training
        if (sampleid<20):
            outfile_name = "./CIAdata/train/"+str(curr_train)+".png"
            curr_train = curr_train+1
            cv2.imwrite(outfile_name, dummy)
        elif (sampleid<25):
            outfile_name = "./CIAdata/test/"+str(curr_test)+".png"
            curr_test = curr_test+1
            cv2.imwrite(outfile_name, dummy)
        else:
            outfile_name = "./CIAdata/val/"+str(curr_val)+".png"
            curr_val = curr_val+1
            cv2.imwrite(outfile_name, dummy)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


# Command

## Training - About 4-5 minutes per epoch
DATA_ROOT=./datasets/CIAdata name=CIAdata_generation which_direction=AtoB gpu=0 cudnn=0 batchSize=10 save_epoch_freq=5 niter=100 th train.lua

## Testing 

DATA_ROOT=./datasets/CIAdata name=CIAdata_generation which_direction=AtoB gpu=0 cudnn=0 phase=val th test.lua